In [129]:
from API.models import *
restaurant=Restaurant.objects.filter(id=5)
reviewsByUser=User.objects.get(id=18)
print(reviewsByUser.id)
#c=RecommendedRate(user_id=1, restaurant_id=1,rating=5)
#c.rating
#reviewsByUser.RecommendedRate
#gAll=gathering.objects.all()
rr=reviewsByUser.recommendedrate_set.all().order_by('-rating')
#posts_lists = map(lambda post: project.post, rr)
#print(posts_lists.values())
mkey=list(rr.values_list('gathering_id',flat=True))
print(mkey)
clauses = ' '.join(['WHEN id=%s THEN %s' % (pk, i) for i, pk in enumerate(mkey)])
ordering = 'CASE %s END' % clauses
queryset = Gathering.objects.filter(pk__in=mkey).extra(
           select={'ordering': ordering}, order_by=('ordering',))
print(queryset)

Gathering.objects.filter(id__in=rr.values('gathering_id'))
#g=Gathering.objects.get(id=27)
#r=g.recommend.get(id=207)
#r.gathering.id

18
[19, 20, 21, 25, 28, 22, 26, 27]
<QuerySet [<Gathering: binno together!>, <Gathering: Delicious!>, <Gathering: First gathering>, <Gathering: qfgggfffdfffff>, <Gathering: uhjkkkkkj>, <Gathering: Tesing>, <Gathering: gsjsbjsjs>, <Gathering: gugjjjj>]>


<QuerySet [<Gathering: binno together!>, <Gathering: Delicious!>, <Gathering: First gathering>, <Gathering: Tesing>, <Gathering: qfgggfffdfffff>, <Gathering: gsjsbjsjs>, <Gathering: gugjjjj>, <Gathering: uhjkkkkkj>]>

In [130]:
reviewsByUser=Review.objects.filter(user=13)
for review in reviewsByUser:
	otherReviews=review.restaurant.review_set.exclude(id=review.id)
	for otherReview in otherReviews:
		print(review.rating-otherReview.rating)
	print('diff')		

-1
-2
-2
diff
3
diff


In [131]:
import pandas as pd
import numpy as np

userid=list(User.objects.all().order_by('id').values_list('id', flat=True))
restaurantid=list(Restaurant.objects.all().values_list('id', flat=True))

rating = pd.DataFrame(index=userid,columns=restaurantid)

allReviews=Review.objects.all()
for review in allReviews:
    rating.at[review.user_id,review.restaurant_id]=review.rating

print(rating)
#print(np.isnan(rating.loc[13,5]))

      1    2    3    4    5    6    7
1   NaN  NaN  NaN  NaN  NaN  NaN  NaN
2   NaN  NaN  NaN  NaN  NaN  NaN  NaN
3   NaN  NaN  NaN  NaN  NaN  NaN  NaN
4   NaN  NaN  NaN  NaN  NaN  NaN  NaN
5   NaN  NaN  NaN  NaN  NaN  NaN  NaN
6   NaN  NaN  NaN  NaN  NaN  NaN  NaN
7   NaN  NaN  NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN  NaN  NaN
9   NaN  NaN  NaN  NaN  NaN  NaN  NaN
10  NaN  NaN  NaN  NaN  NaN  NaN  NaN
11  NaN  NaN  NaN  NaN    5    3    2
12  NaN  NaN  NaN  NaN    3    4  NaN
13  NaN  NaN  NaN  NaN  NaN    2    5
14  NaN  NaN  NaN  NaN  NaN  NaN  NaN
15  NaN  NaN  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN  NaN  NaN
17  NaN  NaN  NaN  NaN  NaN  NaN  NaN
18  NaN  NaN    2  NaN    4    4  NaN


In [132]:
diff = pd.DataFrame(index=restaurantid,columns=restaurantid)
freq = pd.DataFrame(index=restaurantid,columns=restaurantid)
matrixLenght=len(restaurantid)
for i in range(0,matrixLenght-1):
    for j in range(i+1,matrixLenght):
        ratingIndex1=restaurantid[i]
        ratingIndex2=restaurantid[j]
        temp=rating[ratingIndex1].sub(rating[ratingIndex2])
        count=temp.count()
        sum=temp.sum()
        if(count==0):
            diff.at[ratingIndex1,ratingIndex2]=0
            diff.at[ratingIndex2,ratingIndex1]=0
        else:
            diff.at[ratingIndex1,ratingIndex2]=sum/count
            diff.at[ratingIndex2,ratingIndex1]=-sum/count
        freq.at[ratingIndex1,ratingIndex2]=count
        freq.at[ratingIndex2,ratingIndex1]=count

print(diff)
print(freq)
print(diff.loc[1,1]/0)

#print(diff.iloc[1])

     1    2    3    4         5         6    7
1  NaN    0    0    0         0         0    0
2    0  NaN    0    0         0         0    0
3    0    0  NaN    0        -2        -2    0
4    0    0    0  NaN         0         0    0
5    0    0    2    0       NaN  0.333333    3
6    0    0    2    0 -0.333333       NaN   -1
7    0    0    0    0        -3         1  NaN
     1    2    3    4    5    6    7
1  NaN    0    0    0    0    0    0
2    0  NaN    0    0    0    0    0
3    0    0  NaN    0    1    1    0
4    0    0    0  NaN    0    0    0
5    0    0    1    0  NaN    3    1
6    0    0    1    0    3  NaN    2
7    0    0    0    0    1    2  NaN


ZeroDivisionError: float division by zero

In [ ]:
predid=18
restid=7
notnullindex=rating.loc[predid,:].dropna().index
#print(notnullindex)
length=len(notnullindex)
ratingSum=0
freqSum=0
for i in range(0,length):
    ratingSum+=(rating.loc[predid,notnullindex[i]]+diff.loc[restid][notnullindex[i]])*freq.loc[restid][notnullindex[i]]
    freqSum+=freq.loc[restid][notnullindex[i]]
if(freqSum!=0):
    print(ratingSum/freqSum)
else:
    print(0)
#predictedrate=RecommendedRate(user_id=predid, restaurant_id=restid,rating=ratingSum/freqSum)
#predictedrate.save()


In [ ]:
import numpy as np

rating.loc[:,10] = np.nan
rating.loc[14,:]= np.nan
rating.loc[15,11]= 4
rating


In [ ]:

newReviewRest=5
freq.loc[newReviewRest,:] = freq.loc[newReviewRest,:] + 1
freq.loc[:,newReviewRest] = freq.loc[:,newReviewRest] + 1
freq

In [ ]:
restaurantid
matrixLenght=len(restaurantid)
for i in range(0,matrixLenght-1):
    ratingIndex1=restaurantid[i]
    ratingIndex2=restaurantid[j]
    temp=rating[ratingIndex1].sub(rating[ratingIndex2])
    count=temp.count()
    sum=temp.sum()
    diff.at[ratingIndex1,ratingIndex2]=sum/count
    diff.at[ratingIndex2,ratingIndex1]=-sum/count


In [ ]:
RecommendedRate.objects.all()

In [ ]:
from API.recommendation import *
s=SlopeOne()
s.predict(12,5)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

print(freq)
agg = AgglomerativeClustering(n_clusters=3, affinity='precomputed',linkage='average')
print(agg.fit_predict(freq.fillna(0)))

In [152]:
profileList=list(Profile.objects.all().order_by('user_id').values_list('user_id', flat=True))

from django_pandas.io import read_frame
qs = Profile.objects.all()
profileTable = read_frame(qs,fieldnames=['user_id','dob', 'location', 'gender']).set_index('user_id')
#print(profileTable)

locationDistanceMatrix=pd.DataFrame(index=profileList,columns=profileList)
#print(profileTable.iloc[1]['location'])

region=['Hong Kong Island','Hong Kong Island','Hong Kong Island','Hong Kong Island',
        'Kowloon','Kowloon','Kowloon','Kowloon','Kowloon','Kowloon',
        'New Territories','New Territories','New Territories','New Territories','New Territories','New Territories',
       'New Territories','New Territories']
district=['Central and Western','Eastern','Southern','Wan Chai',
          'Sham Shui Po','Kowloon City','Kwun Tong','Wong Tai Sin','Yau Tsim Mong',
          'Islands','Kwai Tsing','North','Sai Kung','Sha Tin','Tai Po','Tsuen Wan','Tuen Mun','Yuen Long']

regionNameList=['Hong Kong Island','Kowloon','New Territories']
districtRegionRelationship=pd.DataFrame(index=district)
districtRegionRelationship['region']=region
#print(districtRegionRelationship.loc['Yuen Long'].values[0])

regionDistanceMatrix=pd.DataFrame(index=regionNameList,columns=regionNameList)
regionDistanceMatrix.iloc[0,1]=8.1
regionDistanceMatrix.iloc[1,0]=8.1
regionDistanceMatrix.iloc[0,2]=16.2
regionDistanceMatrix.iloc[2,0]=16.2
regionDistanceMatrix.iloc[1,2]=12.7
regionDistanceMatrix.iloc[2,1]=12.7
regionDistanceMatrix=regionDistanceMatrix.fillna(0)
print(regionDistanceMatrix)

matrixLenght=len(profileList)
for i in range(0,matrixLenght-1):
    for j in range(i+1,matrixLenght):
        #print("i={0},j={1}".format(i, j))
        myIndex=profileList[i]
        otherIndex=profileList[j]
        myLocation=profileTable.loc[myIndex]['location']
        otherLocation=profileTable.loc[otherIndex]['location']
        myRegion=districtRegionRelationship.loc[myLocation].values[0]
        #print(myRegion)
        otherRegion=districtRegionRelationship.loc[otherLocation].values[0]
        #print(regionDistanceMatrix.loc[myRegion][otherRegion])
        #print(regionDistanceMatrix.iloc[0][1])
        locationDistanceMatrix.loc[myIndex,otherIndex]=regionDistanceMatrix.loc[myRegion,otherRegion]
        locationDistanceMatrix.loc[otherIndex,myIndex]=regionDistanceMatrix.loc[myRegion,otherRegion]

locationDistanceMatrix.fillna(0, inplace=True)
locationDistanceMatrix_norm = ((locationDistanceMatrix - locationDistanceMatrix.values.min()) / 
                               (locationDistanceMatrix.values.max() - locationDistanceMatrix.values.min()))
print(locationDistanceMatrix)
#print(locationDistanceMatrix)
#print(df.iloc[:]['location'])


                  Hong Kong Island  Kowloon  New Territories
Hong Kong Island               0.0      8.1             16.2
Kowloon                        8.1      0.0             12.7
New Territories               16.2     12.7              0.0
      6     7     8     9     10    11    12    13    14    15    16    17  \
6    0.0   0.0   0.0  12.7   0.0  12.7   0.0  12.7  12.7  12.7   0.0   0.0   
7    0.0   0.0   0.0  12.7   0.0  12.7   0.0  12.7  12.7  12.7   0.0   0.0   
8    0.0   0.0   0.0  12.7   0.0  12.7   0.0  12.7  12.7  12.7   0.0   0.0   
9   12.7  12.7  12.7   0.0  12.7   0.0  12.7   0.0   0.0   0.0  12.7  12.7   
10   0.0   0.0   0.0  12.7   0.0  12.7   0.0  12.7  12.7  12.7   0.0   0.0   
11  12.7  12.7  12.7   0.0  12.7   0.0  12.7   0.0   0.0   0.0  12.7  12.7   
12   0.0   0.0   0.0  12.7   0.0  12.7   0.0  12.7  12.7  12.7   0.0   0.0   
13  12.7  12.7  12.7   0.0  12.7   0.0  12.7   0.0   0.0   0.0  12.7  12.7   
14  12.7  12.7  12.7   0.0  12.7   0.0  12.7   0.0   0